# **Step-1:** Importing Required Libraries

In [ ]:
!pip3 install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import glob
import cv2
from keras.models import Model, Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
import os
import seaborn as sns
from keras.applications.efficientnet import EfficientNetB0
from sklearn.ensemble import VotingClassifier

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Step-2:** Performing Preprocessing

In [ ]:
import sklearn
from sklearn.model_selection import train_test_split

#Capture training data and labels into respective lists
images = []
labels = []



for directory_path in glob.glob("/content/drive/MyDrive/NIT Durgapur Internship 3/Dataset_Modified/*"):
   data_split=os.path.split(directory_path)
   label=data_split[-1]
   print(label)
   for img_path in glob.glob(os.path.join(directory_path, "*.jpeg")):

        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (128, 128))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        img=np.asarray(img)
        img=(img/126.5)
        images.append(img)
        labels.append(label)


X_train, X_test, Y_train, Y_test = train_test_split(images, labels, train_size = 0.7, random_state=1, shuffle = True)



Hand
BreastMRI
ChestCT
CXR
AbdomenCT
HeadCT


In [ ]:
#Convert lists to arrays
test_images = np.array(X_test)
test_labels = np.array(Y_test)


In [ ]:
train_images = np.array(X_train)
train_labels = np.array(Y_train)


In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(test_labels)
test_labels_encoded = le.transform(test_labels)
le.fit(train_labels)
train_labels_encoded = le.transform(train_labels)

In [ ]:
#Split data into test and train datasets (already split but assigning to meaningful convention)
x_train, y_train, x_test, y_test = train_images, train_labels_encoded, test_images, test_labels_encoded


In [ ]:
y_train[10:20]

array([2, 0, 3, 2, 0, 4, 3, 3, 3, 2])

In [ ]:
x_train.shape

(18914, 128, 128, 3)

In [ ]:
 # Normalize pixel values to between 0 and 1

In [ ]:
labels=np.unique(y_test)
labels

array([0, 1, 2, 3, 4, 5])

In [ ]:
#One hot encode y values for neural network.
from tensorflow.keras.utils import to_categorical
y_train_one_hot = to_categorical(y_train)
y_test_one_hot = to_categorical(y_test)

# **Step-3:** Feature Extraction using VGG16

In [ ]:
efficientNetB0_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(128,128, 3))

16705208/16705208 [==============================] - 0s 0us/step


In [ ]:
#Make loaded layers as non-trainable. This is important as we want to work with pre-trained weights
for layer in efficientNetB0_model.layers:
	layer.trainable = False

efficientNetB0_model.summary()  #Trainable parameters will be 0

Model: "efficientnetb0"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 128, 128, 3)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 128, 128, 3)  7           ['rescaling[0][0]']              
                                                                                                  
 tf.math.truediv (TFOpLambda)   (None, 128, 128, 3)  0           ['normalization[0][0

In [ ]:
#Now, let us use features from convolutional network for RF
feature_extractor_efficientNet=efficientNetB0_model.predict(x_train)

In [ ]:
fe_efficientNet_test=efficientNetB0_model.predict(x_test)


In [ ]:
features_efficientNet = feature_extractor_efficientNet.reshape(feature_extractor_efficientNet.shape[0], -1)

In [ ]:
f_efficientNet_test= fe_efficientNet_test.reshape(fe_efficientNet_test.shape[0],-1)

In [ ]:
print(f_efficientNet_test.shape)

# **Step-4:** Classifications using Machine Learning Models

## **Step-4.1:** Classification using K-Nearest Neighbours

In [ ]:
import math
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn-whitegrid')

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [ ]:
knn = KNeighborsClassifier(n_neighbors=10)

In [ ]:

knn.fit(features_efficientNet , y_train)
y_pred=knn.predict(f_efficientNet_test)

In [ ]:
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix , plot_confusion_matrix , precision_score,   recall_score ,  f1_score

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

### Confusion Matrix
from sklearn.metrics import confusion_matrix

## **Step-4.2:** Classification using Logistic Regression

In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [ ]:
# define the multinomial logistic regression model
lrm = LogisticRegression(multi_class='multinomial', solver='lbfgs')

In [ ]:
lrm.fit(features_efficientNet , y_train)
y_pred_lr=lrm.predict(f_efficientNet_test)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


## **Step-4.3:** Classification using Random Forest

In [ ]:
#RANDOM FOREST
from sklearn.ensemble import RandomForestClassifier
RF_model = RandomForestClassifier(n_estimators = 50, random_state = 42)

In [ ]:
# Train the model on training data
RF_model.fit(features_efficientNet, y_train) #For sklearn no one hot encoding

RandomForestClassifier(n_estimators=50, random_state=42)

In [ ]:
#Now predict using the trained RF model.
prediction_RF = RF_model.predict(f_efficientNet_test)
#Inverse le transform to get original label back.
prediction_RF = le.inverse_transform(prediction_RF)

## **Step-4.4:** Classification using XGBoost




In [ ]:
!pip install xgboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import xgboost as xgb

In [ ]:
#xgb_classifier = xgb.XGBClassifier(tree_method = "gpu_hist")

In [ ]:
xgb_classifier = xgb.XGBClassifier()

In [ ]:
xgb_classifier.fit(features_efficientNet, y_train)

In [ ]:
predictions = xgb_classifier.predict(f_efficientNet_test)
prediction_XG = le.inverse_transform(predictions)

## **Step-4.5:** Classification using LightGBM


In [ ]:
import lightgbm as lgb

In [ ]:
lightgbm_classifier =lgb.LGBMClassifier()

In [ ]:
lightgbm_classifier.fit(features_efficientNet, y_train)

LGBMClassifier()

In [ ]:
predictions = lightgbm_classifier.predict(f_efficientNet_test)
prediction_lightgbm = le.inverse_transform(predictions)

## **Step-4.6:** Classification using SVC- Support Vector Classifier

In [ ]:
from sklearn.svm import SVC

In [ ]:
svc = SVC(C=0.65, random_state=0, kernel='rbf')

In [ ]:
svc.fit(features_efficientNet, y_train)

In [ ]:
predictions = svc.predict(f_efficientNet_test)
prediction_SVC = le.inverse_transform(predictions)

In [ ]:
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix , plot_confusion_matrix , precision_score,   recall_score ,  f1_score

## **Step-6:** Classification using Ensemble Model

### Ensemble Model-1


In [ ]:
final_model = VotingClassifier(estimators=[('rf', RF_model), ('xgb', xgb_classifier),('knn',knn),('svc',svc),('lr',lrm)], voting='hard')


In [ ]:
final_model.fit(features_efficientNet, y_train)

In [ ]:
predictions = final_model.predict(f_efficientNet_test) # X_test_features
prediction_final = le.inverse_transform(predictions)

### Ensemble Model-2

In [ ]:
final_model_2 = VotingClassifier(estimators=[('rf', RF_model), ('lgbm', lightgbm_classifier),('knn',knn),('svc',svc),('lr',lrm)], voting='hard')


In [ ]:
final_model_2.fit(features_efficientNet, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


VotingClassifier(estimators=[('rf',
                              RandomForestClassifier(n_estimators=50,
                                                     random_state=42)),
                             ('lgbm', LGBMClassifier()),
                             ('knn', KNeighborsClassifier(n_neighbors=10)),
                             ('svc', SVC(C=0.65, random_state=0)),
                             ('lr',
                              LogisticRegression(multi_class='multinomial'))])

In [ ]:
predictions = final_model_2.predict(f_efficientNet_test) # X_test_features
prediction_final = le.inverse_transform(predictions)